In [ ]:
!pip install yfinance==0.1.67
!mamba install bs4==4.10.0 -y
!pip install plotly
!pip install nbformat==4.2.0


You should consider upgrading via the 'C:\Users\malki\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [67]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Define Graphing Function

In [64]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

Use yfinance to Extract Stock Data


In [12]:
tesla = yf.Ticker("TSLA")

Extract stock information and save it in a dataframe named `tesla_data`

In [14]:
tesla_data = tesla.history(period='max')

In [17]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,index,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,0,2010-06-29,1.266667,1.666667,1.169333,1.592667,281494500,0,0.0
1,1,2010-06-30,1.719333,2.028000,1.553333,1.588667,257806500,0,0.0
2,2,2010-07-01,1.666667,1.728000,1.351333,1.464000,123282000,0,0.0
3,3,2010-07-02,1.533333,1.540000,1.247333,1.280000,77097000,0,0.0
4,4,2010-07-06,1.333333,1.333333,1.055333,1.074000,103003500,0,0.0


Use Webscraping to Extract Tesla Revenue Data


In [19]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url).text

In [20]:
soup = BeautifulSoup(html_data, 'html5lib')

In [33]:
tesla_tables = pd.read_html(str(soup), match="Tesla Quarterly Revenue")
tesla_revenue = tesla_tables[0]
tesla_revenue.rename(columns = {'Tesla Quarterly Revenue(Millions of US $)':'Date', 'Tesla Quarterly Revenue(Millions of US $).1':'Revenue'}, inplace = True)


,Date,Revenue
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"


Remove the comma and dollar sign from the `Revenue` column. 


In [36]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

C:\Users\malki\AppData\Local\Temp\ipykernel_17820\349343550.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")


Remove an null or empty strings in the Revenue column.


In [37]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [39]:
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


Use yfinance to Extract Stock Data of GameStop


In [40]:
gameStop = yf.Ticker("GME")

In [41]:
gme_data = gameStop.history(period='max')

In [42]:
gme_data.reset_index(inplace=True)

Use Webscraping to Extract GME Revenue Data


In [43]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data = requests.get(url).text

In [44]:
soup_gme = BeautifulSoup(html_data, 'html5lib')

In [47]:
gme_revenue = pd.read_html(str(soup_gme), match = 'GameStop Quarterly Revenue')[0]
gme_revenue.rename(columns = {'GameStop Quarterly Revenue(Millions of US $)':'Date', 'GameStop Quarterly Revenue(Millions of US $).1':'Revenue'}, inplace = True)

Remove the comma and dollar sign from the `Revenue` column. 

In [52]:
gme_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

C:\Users\malki\AppData\Local\Temp\ipykernel_17820\3659719150.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  gme_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")


Remove an null or empty strings in the Revenue column.


In [54]:
gme_revenue.dropna(inplace=True)

gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

In [55]:
gme_revenue.tail()

,Date,Revenue
48,2008-04-30,31
49,2008-01-31,28
50,2007-10-31,21
52,2007-04-30,46
53,2007-01-31,27


Plot Tesla Stock Graph


In [ ]:
make_graph(stock_data=tesla_data, revenue_data=tesla_revenue, stock='Tesla')

Plot GameStop Stock Graph


In [ ]:
make_graph(stock_data=gme_data, revenue_data=gme_revenue, stock='GameStop')